# Example 2


### Identifying the reaction energy profile of multiple Diels-Alder reaction

This workflow includes:

i) CREST conformer sampling \
ii) Gaussian geometry optimizations and frequency calcs (B3LYP/def2TZVP) \
iii) Dixing errors and imaginary frequencies of the output LOG files \
iv) ORCA single-point energy corrections (SPC) using DLPNO-CCSD(T)/def2TZVPP\
v) Boltzmann weighted thermochemistry calculation with GoodVibes at 298.15 K

#### Steps involved in this example

- Step 1: Importing AQME and other python modules
- Step 2: Determining distance and angle constraints for TSs
- Step 3: CSEARCH conformational sampling
- Step 4: Creating Gaussian input files for optimization and frequency with QPREP
- Step 5: Running Gaussian inputs for optimization and frequency calcs externally
- Step 6: QCORR analysis
- Step 7: Resubmission of the new jobs (if any)
- Step 8: Creating DLPNO input files for ORCA single-point energy calculations
- Step 9: Running ORCA inputs for single point energy calcs externally
- Step 10: Calculating PES with goodvibes

###  Step 1: Importing AQME and other python modules

In [ ]:
import os, glob, subprocess
import shutil
from pathlib import Path
from aqme.csearch import csearch
from aqme.qprep import qprep
from aqme.qcorr import qcorr
from rdkit import Chem
import pandas as pd

### Step 2: Determining distance and angle constraints for TSs

In [ ]:
smi1 = 'C1=CC=CC1.C1=CC1'
mol1 = Chem.MolFromSmiles(smi1)
mol1 = Chem.AddHs(mol1)
for i,atom in enumerate(mol1.GetAtoms()):
    atom.SetAtomMapNum(i)
smi_new1 = Chem.MolToSmiles(mol1)
print('The new mapped smiles for checking numbers used in constraints is:', smi_new1)

mol1
# distance and angle constraints to include in the CSV file are:
# constraits_dist = [[3,5,2.35],[0,6,2.35]]

In [ ]:
smi1 = 'C1=CC=CC1.C1=CCC1'
mol1 = Chem.MolFromSmiles(smi1)
mol1 = Chem.AddHs(mol1)
for i,atom in enumerate(mol1.GetAtoms()):
    atom.SetAtomMapNum(i)
smi_new1 = Chem.MolToSmiles(mol1)
print('The new mapped smiles for checking numbers used in constraints is:', smi_new1)

mol1
# distance and angle constraints to include in the CSV file are:
# constraits_dist = [[3,6,2.35],[0,5,2.35]]
# this case is a bit special, since a four-membered ring TS is formed, and if numbers 5 and 6
# are exchanged, the TS won't be found (i.e. with [[3,5,2.35],[0,6,2.35]])

In [ ]:
smi1 = 'C1=CC=CC1.C1=CCCC1'
mol1 = Chem.MolFromSmiles(smi1)
mol1 = Chem.AddHs(mol1)
for i,atom in enumerate(mol1.GetAtoms()):
    atom.SetAtomMapNum(i)
smi_new1 = Chem.MolToSmiles(mol1)
print('The new mapped smiles for checking numbers used in constraints is:', smi_new1)

mol1
# distance and angle constraints to include in the CSV file are:
# constraits_dist = [[3,5,2.35],[0,6,2.35]]

### Step 3: CSEARCH conformational sampling

In [ ]:
data = pd.read_csv('example2.csv') # read the CSV file with SMILES strings and constraints for TSs (from Step 2)

csearch(input='example2.csv',program='crest',cregen=True,
        cregen_keywords='--ethr 0.1 --rthr 0.2 --bthr 0.3 --ewin 1')

### Step 4: Creating Gaussian input files for optimization and frequency with QPREP

In [ ]:
program = 'gaussian'
mem='32GB'
nprocs=16

# COM files for the TSs
sdf_TS_files = glob.glob('CSEARCH/TS*crest.sdf')
qm_input_TS = 'B3LYP/def2tzvp opt=(ts,calcfc,noeigen,maxstep=5) freq=noraman'
qprep(files=sdf_TS_files,program=program,qm_input=qm_input_TS,mem=mem,nprocs=nprocs)

# COM files for intermediates, reagents and products
sdf_INT_files = glob.glob('CSEARCH/D*.sdf') + glob.glob('CSEARCH/P*.sdf')
qm_input_INT = 'B3LYP/def2tzvp opt freq=noraman'

qprep(files=sdf_INT_files,program=program,qm_input=qm_input_INT,mem=mem,nprocs=nprocs)

### Step 5: Running Gaussian inputs for optimization and frequency calcs externally

In [ ]:
# Run the generated COM files (in the QCALC folder) with Gaussian

### Step 6: QCORR analysis

In [ ]:
qcorr(files='QCALC/*.log',freq_conv='opt=(calcfc,maxstep=5)',mem=mem,nprocs=nprocs)

### Step 7: Resubmission of unsuccessful calculations (if any) with suggestions from AQME

In [ ]:
# Run the generated COM files (in fixed_QM_inputs, shown as resub_path in Step 8) with Gaussian

### Step 8: QCORR analysis of unsuccessful calculations (if any) from Step 7

In [ ]:
resub_path = 'QCALC/failed/run_1/fixed_QM_inputs' # LOG files from Step 6
log_files_resub = f'{resub_path}/*.log'

# this QCORR analysis skips the freq_conv since we noticed that thermochemistry data barely change after one freq_conv correction
# if there are no files that failed, you will see a PATH error
qcorr(files=log_files_resub,isom_type='com',isom_inputs=resub_path,nprocs=12,mem='24GB')

### Step 9: Creating DLPNO input files for ORCA single-point energy calculations

In [ ]:
program = 'orca'
mem='16GB'
nprocs=8

qm_files = os.getcwd()+'/QCALC/success/*.log' # LOG files from Steps 6 and 8
destination =  os.getcwd()+'/SP' # folder where the ORCA output files are generated

# keyword lines for ORCA inputs
qm_input ='DLPNO-CCSD(T) def2-tzvpp def2-tzvpp/C\n'
qm_input += '%scf maxiter 500\n'
qm_input += 'end\n'
qm_input += '% mdci\n'
qm_input += 'Density None\n'
qm_input += 'end\n'
qm_input += '% elprop\n'
qm_input += 'Dipole False\n'
qm_input += 'end'

qprep(destination=destination,files=qm_files,program=program, qm_input=qm_input,mem=mem,nprocs=nprocs, suffix='DLPNO')

### Step 10: Running ORCA inputs for single point energy calcs externally

In [ ]:
# Run the generated COM files (in destination, see PATH in Step 9) with ORCA

### Step 11: Calculating PES with goodvibes

In [ ]:
orca_files = os.getcwd()+'/SP/*.out' # folder where the OUT files from Step 10 are generated

# copy all the Gaussian LOG files and the ORCA OUT files into a new folder called GoodVibes_analysis (necessary to apply SPC corrections)
opt_files = glob.glob(qm_files)
spc_files = glob.glob(orca_files)
all_files = opt_files + spc_files

w_dir_main  = Path(os.getcwd())
GV_folder = w_dir_main.joinpath('GoodVibes_analysis')
GV_folder.mkdir(exist_ok=True, parents=True)

for file in all_files:
	shutil.copy(file, GV_folder)

# run GoodVibes
os.chdir(GV_folder)
subprocess.run(['python', '-m', 'goodvibes', '--xyz','--pes', '../pes.yaml','--graph','../pes.yaml','-c','1','--spc', 'DLPNO', '*.log',])
os.chdir(w_dir_main)